In [ ]:
#pip install wordcloud
#!pip uninstall scikit-learn
#!pip install gensim
#!pip install scikit-learn
#!pip install fdb
#!pip install --upgrade nltk

In [ ]:
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('stopwords')
# nltk.download('maxent_ne_chunker')
# nltk.download('words')

## Import Modules

In [1]:
# Import necessary modules
import fdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re
import nltk
import json
import io
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
# Model building library 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU
from keras.initializers import Constant
from keras.layers import Embedding
# # Imbalanced 
# from imblearn.over_sampling import ADASYN
from sklearn.metrics import precision_score


## Connect to database

In [ ]:
hostname = "localhost"
database_path = "C:/eStream/SQLAccounting/DB/BOK OPT SDN BHD.FDB"
username = "SYSDBA"
password = "masterkey"

In [ ]:
# Create a connection
connection = fdb.connect(
    host=hostname,
    database=database_path,
    user=username,
    password=password,
)

In [ ]:
# Create a Cursor object that operates in the context of Connection con:
cur = connection.cursor()

# Execute the SELECT statement:
cur.execute("select * from GL_CBDTL")

# Fetch all records
records = cur.fetchall()

column_names = [desc[0] for desc in cur.description]

# Close the cursor and connection
cur.close()

cbdt = pd.DataFrame(records, columns=column_names)

In [ ]:
# Create a Cursor object that operates in the context of Connection con:
cur = connection.cursor()

# Execute the SELECT statement:
cur.execute("select * from GL_CB")

# Fetch all records
records = cur.fetchall()

column_names = [desc[0] for desc in cur.description]

# Close the cursor and connection
cur.close()


cb = pd.DataFrame(records, columns=column_names)

In [ ]:
# Create a Cursor object that operates in the context of Connection con:
cur = connection.cursor()

# Execute the SELECT statement:
cur.execute("SELECT t1.CODE, \
                    t1.DESCRIPTION AS CODEDESCRIPTION, \
                    t2.DESCRIPTION AS CATEGORY \
            FROM GL_ACC t1 \
            LEFT JOIN GL_ACC t2 ON t1.PARENT = t2.DOCKEY \
            WHERE t1.PARENT != -1;")

# Fetch all records
records = cur.fetchall()

column_names = [desc[0] for desc in cur.description]

# Close the cursor and connection
cur.close()

connection.close()

acc = pd.DataFrame(records, columns=column_names)

## Data Visualization

In [ ]:
cb.head()

In [ ]:
cbdt.head()

In [ ]:
acc.head()

In [ ]:
cb.isna().sum()

In [ ]:
# Check null value
cbdt.isna().sum()

In [ ]:
# Count the occurrences of each CODE
category_counts = cbdt['CODE'].value_counts()

# Display unique values and their counts
print("Category Counts:")
for category, count in category_counts.items():
    print(f"{category}: {count}")

In [ ]:
from IPython.display import display

# Display unique values and their counts
display(category_counts.reset_index().rename(columns={'index': 'Category', 'Category': 'Count'}))

## Data Preprocessing

In [ ]:
# Filter cashbook column needed to use
selected_columns = ["DOCKEY", "DOCTYPE", "DESCRIPTION"]
filtered_cb = cb[selected_columns]

In [ ]:
# Merge cashbook and cashbook detail by the Dockey
merged_cb = pd.merge(filtered_cb, cbdt, on='DOCKEY')

In [ ]:
merged_cb.head()

In [ ]:
# Filter the doctype to remove OR
merged_cb_categorized = pd.merge(merged_cb, acc, on='CODE', how='inner')

In [ ]:
merged_cb_categorized.head()

In [ ]:
# Get unique values from the specified column
categories = merged_cb_categorized['CATEGORY'].unique()

# Create a dictionary to store individual dataframes
dfs = {}

# Iterate through unique values and create dataframes
for category in categories:
    # Create a new dataframe for each unique value
    new_df = merged_cb_categorized[merged_cb_categorized['CATEGORY'] == category].copy()
    
    # Store the new dataframe in the dictionary
    dfs[f'{category}_df'] = new_df

In [ ]:
for key, value in dfs.items():
    print(f"\n{key}:\n{value}")

In [ ]:
# for key, value in dfs.items():
#     value_counts = value['CODE'].value_counts()
#     values_to_keep = value_counts[value_counts > 5].index
#     dfs[key] = dfs[key][dfs[key]['CODE'].isin(values_to_keep)]

In [ ]:
# to normalize word by removing punctuation, numbers and convert them into same case
def remove_punctuation_and_numbers(text):
    
    # Remove punctuation
    text_no_punct = re.sub(r'[^\w\s]', ' ', text)

    # Remove numbers
    text_no_nums = re.sub(r'\d', ' ', text_no_punct)
    
    # lower text
    text_lower = text_no_nums.lower()
    
    return text_lower

In [ ]:
for key, df in dfs.items():
    dfs[key]['Cleaned_Description_x'] = df['DESCRIPTION_x'].apply(remove_punctuation_and_numbers)
    dfs[key]['Cleaned_Description_y'] = df['DESCRIPTION_y'].apply(remove_punctuation_and_numbers)

In [ ]:
for key, df in dfs.items():
    print(df)

In [ ]:
# Drop DataFrames with no records
dfs = {key: df.dropna(subset='Cleaned_Description_y') for key, df in dfs.items()}

In [ ]:
def remove_name(row):
    # Remove the content of sentence2 from sentence1 
    final_description = row['Cleaned_Description_y'].replace(row['Cleaned_Description_x'], " ")
    return final_description

In [ ]:
# merged_cb['final_description'] = merged_cb.apply(remove_name, axis=1)
for key, df in dfs.items():
    result_columns = df.apply(remove_name, axis=1)
    df['final_description'] = result_columns

In [ ]:
# Lemmatization
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text)
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(lemmatized_tokens)

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [ ]:
for key, df in dfs.items():
    df['Lemmatized_Description'] = df['final_description'].apply(lemmatize_text)

In [ ]:
# Stop words removal
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

In [ ]:
for key, df in dfs.items():
    df['NoStopWord_Description'] = df['Lemmatized_Description'].apply(remove_stopwords)

In [ ]:
# Drop DataFrames with no records
keys_to_drop = []

dfs = {key: df[df['NoStopWord_Description'] != ''] for key, df in dfs.items()}
for key, df in dfs.items():
    if df.empty:
        # DataFrame is empty, add key to the list for dropping
        keys_to_drop.append(key)
        print(key)
        
for key in keys_to_drop:
    dfs.pop(key, None)

In [ ]:
for key, df in dfs.items():
    print(f"Number of records in {key}: {len(df)}")

In [ ]:
# Show top 50 highest frequency words
import nltk
from nltk.tokenize import word_tokenize
descriptions = []
vocabularies = []
frequencey_dists = []
for key, df in dfs.items():
    description = df.NoStopWord_Description.str.cat(sep=' ')
    descriptions.append(description)
    #function to split text into word
    tokens = word_tokenize(description)
    vocabularies.append(set(tokens))
    frequency_dist = nltk.FreqDist(tokens)
    print(sorted(frequency_dist,key=frequency_dist.__getitem__, reverse=True)[0:50])

In [ ]:
# List to store keys of DataFrames to be dropped
keys_to_drop = []

# Check each DataFrame in the dictionary
for key, df in dfs.items():
    if len(df) < 100:
        # DataFrame has fewer than 100 records, add key to the list for dropping
        keys_to_drop.append(key)

# Drop DataFrames with fewer than 100 records
for key in keys_to_drop:
    dfs.pop(key, None)



## Vectorization

In [ ]:
w2v_models = []
for key, df in dfs.items():
    lemmatized_descriptions = [description.split() for description in df['NoStopWord_Description']]
    print(key)
    w2v_models.append(Word2Vec(lemmatized_descriptions, vector_size = 100, window =5, min_count = 5, workers = 4))

In [ ]:
vocab = []
for w2v_model in w2v_models:
    vocab.append(list(w2v_model.wv.index_to_key))

In [ ]:
tokenizers = []
max_lengths = []
vocab_sizes = []
for key, df in dfs.items():
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(df['NoStopWord_Description'])
    tokenizers.append(tokenizer)


    # pad sequences 
    max_lengths.append(max([len(s.split()) for s in df['NoStopWord_Description']]))

    # define vocabulary sizes
    vocab_sizes.append(len(tokenizer.word_index) + 1)

In [ ]:
vocab_sizes[1]

In [ ]:
# tokenizer_json = tokenizer.to_json()
# with io.open('tokenizer.json', 'w', encoding='utf-8') as f:
#     f.write(json.dumps(tokenizer_json, ensure_ascii=False))

In [ ]:
# le = LabelEncoder()

In [ ]:
Xs = []
ys = []
for key, df in dfs.items():
    Xs.append(df['NoStopWord_Description'])
    ys.append(df['CODE'])

In [ ]:
X_tokens = []
X_pads = []

for i in range(len(Xs)):
    X_token = tokenizers[i].texts_to_sequences(Xs[i])
    X_tokens.append(X_token)
    X_pads.append(pad_sequences(X_token, maxlen = max_lengths[i], padding = "post"))

In [ ]:
word_vec_dicts = []

for i in range(len(vocab)):
    word_vec_dict = {}
    for word in vocab[i]:
        word_vec_dict[word] = w2v_models[i].wv.get_vector(word)
    word_vec_dicts.append(word_vec_dict)

In [ ]:
print(len(tokenizers))
print(len(word_vec_dicts))

In [ ]:
embed_matrixs = []

for i in range(len(tokenizers)):
    embed_matrix = np.zeros(shape = (vocab_sizes[i],100))
    for word, y in tokenizers[i].word_index.items():
        embed_vector = word_vec_dicts[i].get(word)
        if embed_vector is not None:
            embed_matrix[y] = embed_vector
    embed_matrixs.append(embed_matrix)

In [ ]:
# print(y)

In [ ]:
y_dummies = []
for y in ys:
    y_dummies.append(pd.get_dummies(y))

In [ ]:
print(y_dummies)

In [ ]:
# from imblearn.over_sampling import SMOTE
# smote = SMOTE(sampling_strategy='minority', k_neighbors=5)

# x_sm, y_sm = smote.fit_resample(X_pad, y_dummies.to_numpy())

In [ ]:
# print(y)

In [ ]:
str_ys_list = []
for y in ys:
    str_ys_list.append([str(str_y) for str_y in y])

In [ ]:
num_labels = []
for str_ys in str_ys_list:
    num_labels.append(len(np.unique(str_ys)))

In [ ]:
# Split the data into training and testing sets
X_trains = []
X_tests = []
y_trains = []
y_tests = []

for i in range(len(X_pads)):
    X_train, X_test, y_train, y_test = train_test_split(X_pads[i], y_dummies[i], test_size=0.2, random_state=42)
    X_trains.append(X_train)
    X_tests.append(X_test)
    y_trains.append(y_train)
    y_tests.append(y_test)

In [ ]:
print(len(X))

In [ ]:
print(len(x_sm))

In [ ]:
print(len(X_train))
print(y_train)

## Model Building

In [ ]:
EMBEDDING_DIM = 100
models = []

for i in range(len(vocab_sizes)):
    model = Sequential()
    model.add(Embedding(vocab_sizes[i], EMBEDDING_DIM, input_length = max_lengths[i]))
    model.add(LSTM(units = 128, dropout = 0.2, recurrent_dropout = 0.2))

    model.add(Dense(num_labels[i], activation = 'softmax'))

    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    models.append(model)

In [ ]:
for i in range(len(models)):
    print(i)
    models[i].fit(X_trains[i], y_trains[i], batch_size = 64, validation_split = 0.2, epochs = 5)

In [ ]:
# model.save('AccountCodeModel.h5')

In [ ]:
# model = load_model('AccountCodeModel.h5') 

In [ ]:
accuracys = []
for i in range(len(models)):
    score, acc = models[i].evaluate(X_tests[i], y_tests[i], batch_size=64, verbose=2)
    accuracys.append(acc)

## Evaluation

In [ ]:
for key, accuracy in zip(dfs.keys(), accuracys):
    print(f'Test accuracy for {key}: {accuracy}')

In [ ]:
# # Make predictions on the test set
# y_prob = model.predict(X_test)
# y_pred = np.argmax(y_prob, axis=1)

# # Convert the NumPy array to a Python list
# predictions_list = y_pred.tolist()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches

# Sample data (replace with your actual data)
keys = list(dfs.keys())[:10]  # Replace with your actual keys

# Define a color dictionary for each key
color_dict = {'EXPENSES_df': 'red', 'CURRENT ASSETS_df': 'green', 'Trade Creditors_df': 'blue', 'Trade Debtors_df': 'orange', 'DIRECT COST OF PROJECT AND OPERATION_df': 'purple',
              'Other Creditors_df': 'brown', 'NON-CURRENT ASSETS_df': 'pink', 'CURRENT LIABILITIES_df': 'gray', 'Cash At Bank_df': 'cyan', 'OTHER INCOME_df': 'lime'}

# Set the figure size
plt.figure(figsize=(20, 15))

# Ensure consistent lengths of keys and accuracies
min_length = min(len(keys), len(accuracies))
keys = keys[:min_length]
accuracys = accuracys[:min_length]

# Create a custom colormap based on the color dictionary with a default color
default_color = 'gray'
cmap = ListedColormap([color_dict.get(key, default_color) for key in keys])

# Plotting the bar chart with colormap
bars = plt.bar(keys, accuracys, color=cmap(range(len(keys))))
plt.xlabel('DataFrames')
plt.ylabel('Test Accuracy')
plt.title('Test Accuracy for Each DataFrame')

# Create a legend with color patches
legend_patches = [mpatches.Patch(color=color_dict[key], label=key) for key in keys]
plt.legend(handles=legend_patches, title='DataFrame Keys', loc='upper left')

plt.ylim(0, 1)  # Set y-axis limit between 0 and 1 for accuracy values
plt.show()